# Set variable value using Chen-Scott estimation

In [1]:
#Author Jinzhou Yao
#Last Edit Date 4/8/2019 
import numpy as np
alpha1=1.8341
mu1=0.05148
sigma1=0.1543
lambda1=-0.1253
alpha2=0.005212
mu2=0.03083
sigma2=0.06689
lambda2=-0.0665
#Here I assume T=1 to simplify calculation, T represent bond's maturity and it 
#can be any reasonable value
T = 1
t = 0
y1 = 0.015
y2 = 0.01
N=470

# Calculate gamma, A and B

In [2]:
gamma1=np.sqrt((alpha1+lambda1)**2+2*sigma1**2)
gamma2=np.sqrt((alpha2+lambda2)**2+2*sigma2**2)
A1=pow((2*gamma1*np.exp((alpha1+lambda1+gamma1)*(T-t)/2))/(((alpha1+lambda1+gamma1)*(np.exp(gamma1*(T-t))-1))+2*gamma1),2*alpha1*mu1/(sigma1**2))
A2=pow((2*gamma2*np.exp((alpha2+lambda2+gamma2)*(T-t)/2))/(((alpha2+lambda2+gamma2)*(np.exp(gamma2*(T-t))-1))+2*gamma2),2*alpha2*mu2/(sigma2**2))
B1=2*(np.exp(gamma1*(T-t))-1)/((alpha1 + lambda1+gamma1)*(np.exp(gamma1 * (T-t))-1)+2*gamma1)
B2=2*(np.exp(gamma2*(T-t))-1)/((alpha2 + lambda2+gamma2)*(np.exp(gamma2 * (T-t))-1)+2*gamma2)

# Now we can calculate P and a(k)

In [3]:
P = A1*A2*np.exp(-B1*y1-B2*y2)
#Suppose the coupon rate is 0.05 and it is paid at the end of the year(the easiest situation to be calculated).
#Of course we can assume coupon is paid semi-annually or other different situation but basically the calculation
#process is the same except we need to use the multi-time form formula of a(k)
c=0.05
a1=-c*B1*P-B1*P
a2=-c*B2*P-B2*P

# To get VaR(5%), the only thing left to be calculate here is v(k)

In [4]:
#Using Monte Carlo simulation
dy1 = list()
dy2 = list()
dt=1/52
np.random.seed(0)
for i in range(N):
    #Suppose weekly interest rate
    dw = np.random.normal(0,dt)
    dy1_ = alpha1* (mu1-y1)*dt + sigma1*np.sqrt(y1)*dw
    dy2_ = alpha2* (mu2-y2)*dt + sigma2*np.sqrt(y2)*dw
    dy1.append(dy1_)
    dy2.append(dy2_)
    y1=y1+dy1_
    y2=y2+dy2_
v1 = np.std(dy1)
v2 = np.std(dy2)
#***********************************************************
# #Or we can approximately use the formula below as v1,v2
# #However, this is not accurate since y1,y2 change along time
# v1 = sigma1*y1
# v2 = sigma2*y2

# Put all parameters into matrix B and V

In [5]:
B=np.array([[-P*B1,a1],
            [-P*B2,a2]])
V=np.array([[v1**2,0],
          [0,v2**2]])

# Finally we can calculate VaR(5%) of this portfolio

In [6]:
#Suppose we invest 1 unit in P and 1 unit in Π
n=np.array([1,1])
var_covar=np.matmul(B.T,V).dot(B)
VaR=1.645*np.sqrt(np.matmul(n.T,var_covar).dot(n))
print('VaR for this portfolio is ',VaR)

VaR for this portfolio is  0.0011466236486714252
